In [7]:
import json
import numpy as np
import matplotlib.pyplot as plt

with open('input.json', 'r') as inputFile:
    datas = json.load(inputFile)
    datas = {key:np.array(data) for key, data in datas.items()}
x1, y1 = datas['x1'], datas['y1']
x2, y2 = datas['x2'], datas['y2']
x3, y3 = datas['x3'], datas['y3']

In [28]:
import matplotlib.colors as mcolors

colors = list(mcolors.TABLEAU_COLORS.keys())
mcolors.TABLEAU_COLORS[colors[0]]

'#1f77b4'

In [9]:
import matplotlib as mpl 
colors = list(mpl.colormaps.values())
class GMM_predictor(object):
    def __init__(self, x: np.ndarray, y: np.ndarray):
        self.x = x
        self.y = y
        self.classes = np.unique(self.y)
        self.colors = colors[:self.classes]
        self.num_class = len(self.classes)
    def draw_points(self):
        for p_x, p_y  in zip(self.x, self.y()

plt.figure()
plt.xlabel('X')
plt.ylabel('y')



plt.scatter(x1[:,0], x1[:,1], s=2, label="Origin")



# ax.scatter(x0, x1, self.y, c=self.y, cmap='Reds', marker='o', label="Origin")
# new_x = np.arange(min(self.x), max(self.x), 0.001)
# new_y = self.predict(new_x)
# plt.scatter(new_x, new_y, c='r',s=0.5, label="Predicted")
# plt.legend()
plt.show()

IndentationError: expected an indented block (3276060545.py, line 4)

In [ ]:
from sklearn.mixture import GMM
gmm = GMM(n_components=4).fit(X)
labels = gmm.predict(X)

In [4]:
from sklearn.mixture import GMM

ImportError: cannot import name 'GMM' from 'sklearn.mixture' (c:\Users\Eonian\Anaconda3\lib\site-packages\sklearn\mixture\__init__.py)